<a href="https://colab.research.google.com/github/leah-apking/project3.1/blob/main/PKE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Install & Import Dependencies

In [99]:
pip install git+https://github.com/boudinfl/pke.git

  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-pnadywoi
  Running command git clone --filter=blob:none --quiet https://github.com/boudinfl/pke.git /tmp/pip-req-build-pnadywoi
  Resolved https://github.com/boudinfl/pke.git to commit 69871ffdb720b83df23684fea53ec8776fd87e63
  Preparing metadata (setup.py) ... done


In [100]:
!pip install -q transformers

In [101]:
!python -m spacy download en_core_web_sm

2023-10-20 16:03:15.622857: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 52.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [102]:
import pandas as pd
import numpy as np
import pke
from transformers import pipeline

## Pull NPS Feedback Data

In [103]:
month='09'
nps_path = f"{month}.ClientNPSSurveyFeedback.csv"
nps_data = pd.read_csv(nps_path)

In [104]:
nps = nps_data[['FullName', 'LoanNumber', 'CompanyName', 'NetPromoterLabel', 'Rating',
       'Comments', 'PerformedBy']]
nps1 = nps.rename(columns={'FullName':'ClientName', 'PerformedBy':'TeamMemberName'})
nps1["Comment"] = nps1.Comments.map(lambda x: 'False' if x == '0' else 'True')

In [105]:
nps2 = nps1.loc[nps1['Comment'] == 'True']
comments = nps2[['LoanNumber', 'Comments']]
comments1 = comments.dropna()

In [241]:
comments['WordCount'] = comments['Comments'].str.count(' ') + 1
nps_short = comments.loc[(comments['WordCount'] > 8) & (comments['WordCount'] < 10)]
nps_short.count()

<ipython-input-241-f3af2cf171c7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['WordCount'] = comments['Comments'].str.count(' ') + 1


LoanNumber    14
Comments      14
WordCount     14
dtype: int64

In [ ]:
comments['WordCount'] = comments['Comments'].str.count(' ') + 1
nps_med = comments.loc[(comments['WordCount'] >= 10) & (comments['WordCount'] <= 30)]
nps_med.count()

<ipython-input-203-4db698adc995>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['WordCount'] = comments['Comments'].str.count(' ') + 1


LoanNumber    110
Comments      110
WordCount     110
dtype: int64

In [ ]:
comments['WordCount'] = comments['Comments'].str.count(' ') + 1
nps_long = comments.loc[comments['WordCount'] > 30]
nps_long.count()

<ipython-input-203-4db698adc995>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['WordCount'] = comments['Comments'].str.count(' ') + 1


LoanNumber    110
Comments      110
WordCount     110
dtype: int64

## Extract Sentiment and KeyPhrases from Comments

### "Short" Comments (WordCount < 10)

In [204]:
# phraselist1= []
# # scorelist = []

# def pke_extract1(text):
#   extractor = pke.unsupervised.TopicRank()

#   extractor.load_document(input=text, language='en')

#   extractor.candidate_selection()

#   extractor.candidate_weighting()

#   keyphrases = extractor.get_n_best(n=1, stemming=False)

#   for i, (candidate, score) in enumerate(keyphrases):

#     phraselist1.append(candidate)
#     # scorelist.append(score)

# for n in range(len(nps_short)):
#     text = nps_short.iloc[n,1]
#     pke_extract1(text)

In [246]:
sentimentlist1 = []

ranklist1 = []
phraselist1= []
# scorelist1 = []

def sentiment1(text):

  sentiment_pipeline = pipeline("sentiment-analysis")

  sentiments = sentiment_pipeline(text)

  for s, data in enumerate(sentiments):

    sentimentlist1.append(data)

def pke_extract1(text):

  extractor = pke.unsupervised.TopicRank()

  extractor.load_document(input=text, language='en')

  extractor.candidate_selection()

  extractor.candidate_weighting()

  keyphrases1 = extractor.get_n_best(n=1, stemming=False)

  for i, (candidate, score) in enumerate(keyphrases1):

    ranklist1.append(i)
    phraselist1.append(candidate)
    # scorelist1.append(score)

for n in range(len(nps_short)):
    text = nps_short.iloc[n,1]
    sentiment1(text)
    pke_extract1(text)



No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-englis

In [250]:
words1 = pd.DataFrame(list(zip(phraselist1, ranklist1)), columns = ['1st', 'Rank'])
words1

,1st,Score
0,loans,0
1,support,0
2,great job,0
3,account executive,0
4,great job,0
5,speed,0
6,cheers,0
7,closing,0
8,clean,0
9,ae beau,0


### "Medium" Comments (WordCount 10-30)

In [ ]:
sentimentlist3 = []

ranklist3 = []
phraselist3= []
scorelist3 = []

def sentiment3(text):

  sentiment_pipeline = pipeline("sentiment-analysis")

  sentiments = sentiment_pipeline(text)

  for s, data in enumerate(sentiments):

    sentimentlist3.append(data)

def pke_extract3(text):
  extractor = pke.unsupervised.TopicRank()

  extractor.load_document(input=text, language='en')

  extractor.candidate_selection()

  extractor.candidate_weighting()

  keyphrases3 = extractor.get_n_best(n=3, stemming=False)

  for i, (candidate, score) in enumerate(keyphrases3):

    ranklist3.append(i)
    phraselist3.append(candidate)
    scorelist3.append(score)

for n in range(len(nps_med)):
    text = nps_med.iloc[n,1]
    sentiment3(text)
    pke_extract3(text)

### "Long" Comments (WordCount > 30)

In [ ]:
sentimentlist5 = []

ranklist5 = []
phraselist5 = []
scorelist5 = []

def sentiment5(text):

  sentiment_pipeline = pipeline("sentiment-analysis")

  sentiments = sentiment_pipeline(text)

  for s, data in enumerate(sentiments):

    sentimentlist5.append(data)

def pke_extract5(text):
  extractor = pke.unsupervised.TopicRank()

  extractor.load_document(input=text, language='en')

  extractor.candidate_selection()

  extractor.candidate_weighting()

  keyphrases5 = extractor.get_n_best(n=5, stemming=False)

  for i, (candidate, score) in enumerate(keyphrases5):

    ranklist5.append(i)
    phraselist5.append(candidate)
    scorelist5.append(score)

for n in range(len(nps_long)):
    text = nps_long.iloc[n,1]
    sentiment5(text)
    pke_extract5(text)

In [208]:
print(phraselist)

['pleasure', 'excellent team work', 'amazing', 'speedy', 'fast process', 'great partnership', 'easy', 'smooth', 'uwm team', 'best company', 'great job', 'amazing', 'entire uwm team', 'fast closing', 'excellent service', 'guys', 'awesome', 'loans', 'excellent job', 'overall', 'support', 'fantastic sr uw', 'fantastic', 'brisk', 'amazing', 'eric campbell', 'consistent', 'best ae eric stein', 'holly compton', 'great help', 'ctc', 'great job', 'quick efficient service', 'awesome job', 'awesome work', 'unmatched services', 'account executive', 'great job', 'spectacular service', 'best ae', 'amazing', 'speed', 'broker', 'great job', 'great communication', 'best', 'loan process easy', 'senior underwriter', 'fine', 'great work', 'cheers', 'knowledgeable', 'smooth transaction', 'love', 'process', 'uwm', 'fantastic sr uw', 'fantastic', 'brisk', 'amazing', 'knowledgeable', 'fast underwriting', 'closing', 'smooth process', 'excellent communiation service', 'loan', 'great', 'best rep', 'thanks', 'an

In [211]:
# list1 = []
# sentimentlist = []

# def sentiment(text):

#   sentiment_pipeline = pipeline("sentiment-analysis")

#   sentiments = sentiment_pipeline(text)

#   for s, data in enumerate(sentiments):
#     list1.append(s)
#     sentimentlist.append(data)

# for n in range(len(nps_comments)):
#     text = nps_comments.iloc[n,1]
#     sentiment(text)

In [212]:
# sentimentlist = []
# list1 = []

# ranklist = []
# phraselist= []
# scorelist = []

# def sentiment(text):

#   sentiment_pipeline = pipeline("sentiment-analysis")

#   sentiments = sentiment_pipeline(text)

#   for s, data in enumerate(sentiments):
#     list1.append(s)
#     sentimentlist.append(data)

# def pke_extract(text):
#   extractor = pke.unsupervised.TopicRank()

#   extractor.load_document(input=text, language='en')

#   extractor.candidate_selection()

#   extractor.candidate_weighting()

#   keyphrases = extractor.get_n_best(n=1, stemming=False)

#   for i, (candidate, score) in enumerate(keyphrases):

#     ranklist.append(i)
#     phraselist.append(candidate)
#     scorelist.append(score)

# for n in range(len(nps_comments)):
#     text = nps_comments.iloc[n,1]
#     sentiment(text)
#     pke_extract(text)

## Build DataFrame

In [194]:
df1 = pd.DataFrame(sentimentlist1)
df3 = pd.DataFrame(sentimentlist3)
df5 = pd.DataFrame(sentimentlist5)


,label,score
0,POSITIVE,0.999392
1,POSITIVE,0.999794
2,POSITIVE,0.999848
3,POSITIVE,0.999876
4,POSITIVE,0.999876
...,...,...
127,NEGATIVE,0.998263
128,NEGATIVE,0.953232
129,NEGATIVE,0.992574
130,NEGATIVE,0.988391


In [201]:
words1 = pd.DataFrame(list(zip(phraselist, scorelist)), columns = ['1st', 'Score'])
words1

,1st,Score
0,pleasure,0.393617
1,excellent team work,1.000000
2,amazing,0.500000
3,speedy,0.393617
4,fast process,0.408888
...,...,...
103,jumbos,0.284205
104,closing day,0.500000
105,better communication,0.500000
106,clear,0.500000


In [181]:
def divide_chunks1(l, n):

    for i in range(0, len(l), n):
        yield l[i:i + n]
n = 1

phrases = list(divide_chunks1(phraselist, n))
columns = ['1st']
keywords_df1 = pd.DataFrame(phrases,columns=columns)
keywords_df1

,1st
0,pleasure
1,great love uwm
2,excellent team work
3,amazing
4,speedy
...,...
150,clear
151,slow process
152,slow process
153,unnecessary touches


In [ ]:
def divide_chunks3(l, n):

    for i in range(0, len(l), n):
        yield l[i:i + n]
n = 3

phrases = list(divide_chunks3(phraselist, n))
columns = ['1st', '2nd', '3rd']
keywords_df3 = pd.DataFrame(phrases,columns=columns)
keywords_df3

In [153]:
def divide_chunks5(l, n):

    for i in range(0, len(l), n):
        yield l[i:i + n]
n = 5

phrases = list(divide_chunks5(phraselist, n))
columns = ['1st', '2nd', '3rd', '4th', '5th']
keywords_df5 = pd.DataFrame(phrases,columns=columns)
keywords_df5

,1st,2nd,3rd,4th,5th
0,uwm,closing team,awesome,easy,process
1,christina plante,jackie berry,awesome,hiccups,couple
2,able,gerry nelson,kyle smith,anthony brikho,nathan smedes
3,garett hess,conditions,succinct,gentleman,prompt
4,briana kent,questions,loan submission,closing team,help
...,...,...,...,...,...
228,fault,loan,disaster,ratings,mistake
229,file,unhelpful,dismissive,complete,rate
230,email,dollars,closing,consistent uw,nice
231,surprise,enote,uwm,agent,builder


In [154]:
# phrases2 = list(divide_chunks(phraselist2, n))

# keywords2_df = pd.DataFrame(phrases2,columns=columns)
# keywords2_df

In [188]:
len(scorelist)

155

In [155]:
df1 = pd.concat([sentiment_df, keywords_df], axis=1)
# df2 = pd.concat([df1, nps_comments], axis=1)
# df2.head()
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   label   263 non-null    object 
 1   score   263 non-null    float64
 2   1st     233 non-null    object 
 3   2nd     233 non-null    object 
 4   3rd     232 non-null    object 
 5   4th     232 non-null    object 
 6   5th     232 non-null    object 
dtypes: float64(1), object(6)
memory usage: 14.5+ KB


In [156]:
df1

,label,score,1st,2nd,3rd,4th,5th
0,POSITIVE,0.832655,uwm,closing team,awesome,easy,process
1,POSITIVE,0.999789,christina plante,jackie berry,awesome,hiccups,couple
2,POSITIVE,0.912482,able,gerry nelson,kyle smith,anthony brikho,nathan smedes
3,POSITIVE,0.999395,garett hess,conditions,succinct,gentleman,prompt
4,NEGATIVE,0.851754,briana kent,questions,loan submission,closing team,help
...,...,...,...,...,...,...,...
258,NEGATIVE,0.999366,NaN,NaN,NaN,NaN,NaN
259,NEGATIVE,0.997779,NaN,NaN,NaN,NaN,NaN
260,NEGATIVE,0.962565,NaN,NaN,NaN,NaN,NaN
261,NEGATIVE,0.998787,NaN,NaN,NaN,NaN,NaN


In [210]:
from google.colab import files

export_path = f"{month}_NPS_Comments_Processed.csv"
comments.to_csv(export_path, encoding = 'utf-8-sig')
files.download(export_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>